# Sleep Efficiency Dataset

## Day 1: EDA and Data Cleaning



In [1]:
# Loading packages that we need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import PowerTransformer, StandardScaler, minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

### Loading & Analyzing the data

In [2]:
# Loading data
sleepy_df = pd.read_csv('Sleep_Efficiency.csv')
sleepy_df.head(3)

,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0


In [3]:
# Analyzing the columns
sleepy_df.columns.unique()

Index(['ID', 'Age', 'Gender', 'Bedtime', 'Wakeup time', 'Sleep duration',
       'Sleep efficiency', 'REM sleep percentage', 'Deep sleep percentage',
       'Light sleep percentage', 'Awakenings', 'Caffeine consumption',
       'Alcohol consumption', 'Smoking status', 'Exercise frequency'],
      dtype='object')

In [4]:
# Analyzing the data type and number of NaNs
sleepy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      452 non-null    int64  
 1   Age                     452 non-null    int64  
 2   Gender                  452 non-null    object 
 3   Bedtime                 452 non-null    object 
 4   Wakeup time             452 non-null    object 
 5   Sleep duration          452 non-null    float64
 6   Sleep efficiency        452 non-null    float64
 7   REM sleep percentage    452 non-null    int64  
 8   Deep sleep percentage   452 non-null    int64  
 9   Light sleep percentage  452 non-null    int64  
 10  Awakenings              432 non-null    float64
 11  Caffeine consumption    427 non-null    float64
 12  Alcohol consumption     438 non-null    float64
 13  Smoking status          452 non-null    object 
 14  Exercise frequency      446 non-null    fl

In [5]:
nancount = sleepy_df.isna().sum()
print(nancount)

ID                         0
Age                        0
Gender                     0
Bedtime                    0
Wakeup time                0
Sleep duration             0
Sleep efficiency           0
REM sleep percentage       0
Deep sleep percentage      0
Light sleep percentage     0
Awakenings                20
Caffeine consumption      25
Alcohol consumption       14
Smoking status             0
Exercise frequency         6
dtype: int64


In [6]:
# How many duplicates are present
print(sleepy_df.duplicated().sum())

0


### Uniforming the columns

In [7]:
#Lower case all the columns
def lower_case_column_names():
    sleepy_df.columns=[i.lower() for i in sleepy_df.columns]
    return sleepy_df

In [8]:
sleepy_df = lower_case_column_names()

In [9]:
# Renaming the columns with underscores 
def rename_columns(sleepy_df):
    sleepy_df.rename(columns={'wakeup time':'wakeup_time', 'sleep duration':'sleep_duration',
                              'sleep efficiency':'sleep_efficiency', 'rem sleep percentage':'rem_sleep_percentage',
                             'deep sleep percentage':'deep_sleep_percentage','light sleep percentage':'light_sleep_percentage',
                             'caffeine consumption':'caffeine_consumption', 'alcohol consumption':'alcohol_consumption',
                             'smoking status':'smoking_status','exercise frequency':'exercise_frequency'}, inplace=True )
    return sleepy_df
sleepy_df=rename_columns(sleepy_df)

sleepy_df.head(3)

,id,age,gender,bedtime,wakeup_time,sleep_duration,sleep_efficiency,rem_sleep_percentage,deep_sleep_percentage,light_sleep_percentage,awakenings,caffeine_consumption,alcohol_consumption,smoking_status,exercise_frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0


In [10]:
# Checking for typos in text columns
sleepy_df['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [11]:
sleepy_df['smoking_status'].unique()

array(['Yes', 'No'], dtype=object)

In [12]:
# Calculating the mean of every column, which contains NaNs. 
# These values will be later used for substituting the NaNs with the respective mean of the respective column

def mean_without_nans(sleepy_df, column_name):

# Get the column as a pandas Series
    column = sleepy_df[column_name]
    
# Initialize a sum (=total) and a count
    total = 0
    count = 0
    
# Loop over the values in the column
    for value in column.values:
        if pd.notnull(value):
            total += value
            count += 1
    
# Return the mean of the non-NaN values
    return total / count if count != 0 else float("nan")


In [13]:
# Call the mean_without_nans function on column with nans
mean_awakenings = mean_without_nans(sleepy_df, 'awakenings')
mean_caffeine_consumption = mean_without_nans(sleepy_df, 'caffeine_consumption')
mean_alcohol_consumption = mean_without_nans(sleepy_df, 'alcohol_consumption')
mean_exercise_frequency = mean_without_nans(sleepy_df, 'exercise_frequency')
print ('mean_awakenings: ', mean_awakenings, \
      'mean_caffeine_consumption: ', mean_caffeine_consumption, \
      'mean_alcohol_consumption: ', mean_alcohol_consumption, \
      'mean_exercise_frequency: ', mean_exercise_frequency)

mean_awakenings:  1.6412037037037037 mean_caffeine_consumption:  23.653395784543324 mean_alcohol_consumption:  1.17351598173516 mean_exercise_frequency:  1.7914798206278026


In [14]:
# Fill nans with the mean. We asume the person consumes an average amount of coffe & alcohol, plus they have 
# an average amount of sport-time and awakenings during the night.

sleepy_df['awakenings'] = sleepy_df[['awakenings']].fillna(mean_awakenings) #imputation 

In [15]:
# Controling if the values are being replaced
sleepy_df['awakenings'].unique()

array([0.       , 3.       , 1.       , 2.       , 4.       , 1.6412037])

In [16]:
sleepy_df['caffeine_consumption'] = sleepy_df[['caffeine_consumption']].fillna(mean_caffeine_consumption)

In [17]:
sleepy_df['caffeine_consumption'].unique()

array([  0.        ,  50.        ,  23.65339578,  25.        ,
        75.        , 200.        , 100.        ])

In [18]:
sleepy_df['alcohol_consumption'] = sleepy_df[['alcohol_consumption']].fillna(mean_alcohol_consumption)

In [19]:
sleepy_df['alcohol_consumption'].unique()

array([0.        , 3.        , 5.        , 1.        , 2.        ,
       1.17351598, 4.        ])

In [20]:
sleepy_df['exercise_frequency'] = sleepy_df[['exercise_frequency']].fillna(mean_exercise_frequency)

In [21]:
sleepy_df['exercise_frequency'].unique()

array([3.        , 1.        , 0.        , 5.        , 2.        ,
       4.        , 1.79147982])

In [22]:
sleepy_df

,id,age,gender,bedtime,wakeup_time,sleep_duration,sleep_efficiency,rem_sleep_percentage,deep_sleep_percentage,light_sleep_percentage,awakenings,caffeine_consumption,alcohol_consumption,smoking_status,exercise_frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.000000,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.000000,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.000000,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.000000,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.000000,3.0,No,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,448,27,Female,2021-11-13 22:00:00,2021-11-13 05:30:00,7.5,0.91,22,57,21,0.0,0.000000,0.0,No,5.0
448,449,52,Male,2021-03-31 21:00:00,2021-03-31 03:00:00,6.0,0.74,28,57,15,4.0,25.000000,0.0,No,3.0
449,450,40,Female,2021-09-07 23:00:00,2021-09-07 07:30:00,8.5,0.55,20,32,48,1.0,23.653396,3.0,Yes,0.0
450,451,45,Male,2021-07-29 21:00:00,2021-07-29 04:00:00,7.0,0.76,18,72,10,3.0,0.000000,0.0,No,3.0


<b/> Change data-type for time-stamps and Bin them

In [ ]:
# Bins the wakeup-time and bed-time
sleepy_df['bins'] = pd.cut(x= slepy_df.bedtime, bins =[ "07:00", 2024])
##years = cars.goupby('bins')['Year'].count()

<b/>Dummies

In [ ]:
categorical = sleepy_df.select_dtypes(include='object') 

In [ ]:
categorical.columns.unique()

In [ ]:
# change data type

#file2_df[['income', 'monthly_premium_auto', 'total_claim_amount']] = file2_df[['income', 'monthly_premium_auto', 'total_claim_amount']].astype(float)
#file2_df.info()

# To do: 

# Bins the time stamp and bed time
## cars['bins'] = pd.cut(x= df.Year, bins =[ 1980, 1990, 2000, 2010, 2020])
##years = cars.goupby('bins')['Year'].count()

# Make dummys 

In [ ]:
# Let us calculate the mean of those columns where we find NaNs
# Columns to work on: 
# Replace the nans with the mean
#df.replace(np.nan, "MEaaAN", inplace = True)

In [ ]:
# To do:
 # verify uniques in dummies #check
 # change categoricals to dummies 
 # bins para wake up and bed time
 # change integers to floats